In [30]:
import glob
import os
import json

file_list = glob.glob(os.path.join("data","json","*.json"))

competitions = list()
for item in file_list:
    with open(item, 'r') as fp:
        competitions.append(json.load(fp))

In [29]:
len(competitions)

38

In [31]:
import pandas as pd
df = pd.DataFrame({
    'competition': [],
    'event': [],
    'rank': [],
    'name': [],
    'total_base_value': [],
    'total_element_score': [],
    'total_component_score': [],
    'total_segment_score':[]
})
count = 0
for competition in competitions:
    for event in competition['event']:
        try:
            detailed_scores = event['detailed_scores']
        except Exception as e:
            print(f"No detailed scores for {event['competition']}: {event['program']}")
        performances = detailed_scores['performances']
        for performance in performances:
            # print(performance['metadata'])
            df_row = [event['competition'],
                      performance['metadata']['program'],
                      performance['metadata']['rank'],
                      performance['metadata']['name'],
                      performance['metadata']['total_base_value'],
                      performance['metadata']['total_element_score'],
                      performance['metadata']['total_component_score'],
                      performance['metadata']['total_segment_score'],
                      ]
            # print(df_row)
            df.loc[count] = df_row
            count += 1

No detailed scores for 11th Annual Snowtown Invitational: 51 Beginner-No Test-Pre-Prelim Character Performan
No detailed scores for 11th Annual Snowtown Invitational: 52 Beginner-Pre-Juvenile Lyrical Pop
No detailed scores for 11th Annual Snowtown Invitational: 53 Preliminary-Pre-Juvenile Character Performance
No detailed scores for 11th Annual Snowtown Invitational: 54 Junior-Senior Character Performance
No detailed scores for 2022 Crossroads FSC Spring-tacular Competition: Junior Men SP
No detailed scores for 2022 Magnolia Open: Intermediate Ladies Excel -- 210
No detailed scores for 2022 Magnolia Open: Pre-Preliminary Boys Excel -- 134
No detailed scores for 2022 Magnolia Open: Senior Ladies Excel -- 213
No detailed scores for 2022 North Shore Open: 137 Junior Men
No detailed scores for 2022 North Shore Open: 154 Junior Men FS
No detailed scores for 2023 Albert Viviani Memorial Competition Deborah B: 10 Adult Bronze Emotional Performance
No detailed scores for 2023 Albert Viviani Me

In [32]:
df_sorted = df.sort_values(by=['event','rank'])
df_sorted

,competition,event,rank,name,total_base_value,total_element_score,total_component_score,total_segment_score
2856,2023 Skate Austin Bluebonnet Open,001 Intermediate Women SP / Short Program,1,"Carol McKinnis, Dallas FSC",17.38,17.07,13.19,29.26
2857,2023 Skate Austin Bluebonnet Open,001 Intermediate Women SP / Short Program,2,"Maria Sidorova, Oklahoma City FSC",13.68,12.81,13.04,25.85
2858,2023 Skate Austin Bluebonnet Open,001 Intermediate Women SP / Short Program,3,"Lola Hagbo, Memorial FSC Of Houston",15.94,13.93,12.82,25.75
2859,2023 Skate Austin Bluebonnet Open,001 Intermediate Women SP / Short Program,4,"Adelina Toy, Dallas FSC",12.86,11.58,13.27,24.85
2860,2023 Skate Austin Bluebonnet Open,001 Intermediate Women SP / Short Program,5,"Kathy Xinyi Xi, SC of Houston",13.48,11.89,12.91,24.30
...,...,...,...,...,...,...,...,...
6052,Morris Open,Silver Solo Pattern Dance (SDS) (18-21) / P2 T...,2,Milly Wasserman,10.00,11.25,19.59,30.84
6053,Morris Open,Silver Solo Pattern Dance (SDS) (18-21) / P2 T...,3,Jacquelyn Lee,10.00,11.50,18.42,29.92
6054,Morris Open,Silver Solo Pattern Dance (SDS) (18-21) / P2 T...,4,Taapti Rana,10.00,10.75,18.84,29.59
6055,Morris Open,Silver Solo Pattern Dance (SDS) (18-21) / P2 T...,5,Ellie Stone,10.00,10.75,17.64,28.39


In [64]:
event_watched = df_sorted[df_sorted['name'].str.contains('Gate City')]
event_watched

,competition,event,rank,name,total_base_value,total_element_score,total_component_score,total_segment_score
1004,2022 Southern CT Open,109 Juvenile Plus Girls Excel / Free Skate,6,"Charlotte Dezotell, Gate City FSC",10.43,10.10,16.57,26.67
1007,2022 Southern CT Open,109 Juvenile Plus Girls Excel / Free Skate,9,"Mia Yakuboff, Gate City FSC",11.00,10.67,14.05,24.72
568,2022 Excel National Festival,11-Excel Juvenile Plus Girls / Free Skate,13,"Mia Yakuboff, Gate City FSC",11.68,11.93,16.35,28.28
777,2022 North Shore Open,120 Preliminary Plus Girls Excel Grp A / Free ...,2,"Alyssa Chan, Gate City FSC",10.48,10.55,10.50,21.05
780,2022 North Shore Open,120 Preliminary Plus Girls Excel Grp A / Free ...,5,"Mallory Lee, Gate City FSC",8.40,8.28,9.19,17.47
784,2022 North Shore Open,121 Preliminary Plus Girls Excel Grp B / Free ...,3,"Kelsey Cochran, Gate City FSC",8.74,7.44,8.58,15.52
822,2022 North Shore Open,128 Juvenile Plus Girls Excel Grp A / Free Skate,1,"Mia Yakuboff, Gate City FSC",13.58,14.34,18.85,33.19
825,2022 North Shore Open,128 Juvenile Plus Girls Excel Grp A / Free Skate,4,"Charlotte Dezotell, Gate City FSC",11.76,11.78,16.78,28.06
1091,2022 Southern CT Open,137 Preliminary Plus Girls Excel Grp A / Free ...,10,"Mallory Lee, Gate City FSC",9.48,8.90,7.66,16.56
1092,2022 Southern CT Open,137 Preliminary Plus Girls Excel Grp A / Free ...,11,"Alyssa Chan, Gate City FSC",8.54,7.83,7.53,14.86


In [65]:
import itertools
plot_df_lt3 = event_watched[event_watched['rank']<=3]
plot_df_gt3 = event_watched[event_watched['rank']>3]
list_lt3 = list(plot_df_lt3['total_base_value'])
list_gt3 = list(plot_df_gt3['total_base_value'])

# plot_df = pd.DataFrame((e for e in itertools.zip_longest(list_lt3,list_gt3)),columns=['top3','rest_of_field'])
plot_df = pd.DataFrame((e for e in itertools.zip_longest(list_lt3,list_gt3)),columns=['top3','rest_of_field'])
plot_df

,top3,rest_of_field
0,10.48,10.43
1,8.74,11.00
2,13.58,11.68
3,22.31,8.40
4,10.50,11.76
5,10.18,9.48
6,8.44,8.54
7,13.03,17.50
8,10.58,9.86
9,13.77,8.76


In [66]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.line(plot_df,y=['top3','rest_of_field'],title='Base Value for Intermediate Ladies')
fig_widget = go.FigureWidget(fig)
fig_widget.show()

In [ ]:
event_watched = df_sorted[df_sorted['event'].str.contains('Preliminary Plus Girls Excel')]
event_watched

In [ ]:
plot_df_lt3 = event_watched[event_watched['rank']<=3]
plot_df_gt3 = event_watched[event_watched['rank']>3]
list_lt3 = list(plot_df_lt3['total_base_value'])
list_gt3 = list(plot_df_gt3['total_base_value'])

# plot_df = pd.DataFrame((e for e in itertools.zip_longest(list_lt3,list_gt3)),columns=['top3','rest_of_field'])
plot_df = pd.DataFrame((e for e in itertools.zip_longest(list_lt3,list_gt3)),columns=['top3','rest_of_field'])
plot_df
fig = px.line(plot_df,y=['top3','rest_of_field'],title='Base Value for Preliminary Plus Girls')
fig_widget = go.FigureWidget(fig)
fig_widget.show()